In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
sns.set()
sns.set(rc={'figure.figsize':(11.7,8.27)})

In [ ]:
df_explo = pd.read_csv("../../../statespace_res.csv")

df_header = pd.read_csv("../../../statespace_header.csv")

In [ ]:
df_explo["schedulable"] = df_explo["schedulable"].astype(bool)

In [ ]:
df_explo["duration_s"] = df_explo["duration"]/1000

In [ ]:
df = df_explo.merge(df_header, left_on="tid", right_on="ts_id")

In [ ]:
df["n_task"] = df["nbt"]

In [ ]:
error = df.groupby("tid")["schedulable"].apply(lambda x: (x.iloc[0] == x).all())
error.loc[~error]

In [ ]:
s = df["U"]<0.6
s &= df["schedulable"]==0
df.loc[s]

In [ ]:
task_palette = sns.color_palette("Set1", int(df["n_task"].unique().shape[0]))

In [ ]:
s = df.shape[0] * [True]
df_plot = df.loc[s]
df_piv = df_plot.copy().pivot(index="tid", columns="search_type", values=["duration_s", "n_visited", "schedulable", "nbt"])
df_piv.columns =list(map(lambda x: "_".join(x), df_piv.columns))


df_piv.rename(columns={"nbt_BFS":"Number of tasks"}, inplace=True)
df_piv.rename(columns={"schedulable_ACBFS":"Schedulable"}, inplace=True)

f, ax = plt.subplots(figsize=(12,12))

sns.scatterplot(data=df_piv, x="n_visited_BFS", y="n_visited_ACBFS", style="Schedulable", hue="Number of tasks", ax=ax, palette=task_palette)
ax.set(xscale="log", yscale="log")
ax.axis('equal')

lims = [
    10**0,
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
]

# now plot both limits against eachother
ax.plot(lims, lims, 'k-', alpha=0.75, zorder=1)

ax.set(xlim=lims, ylim=lims)

ax.set_xlabel("Number of state visited with BFS")
ax.set_ylabel("Number of state visited with ACBFS")

In [ ]:
s = df.shape[0] * [True]
df_plot = df.loc[s]
df_piv = df_plot.copy().pivot(index="tid", columns="search_type", values=["duration_s", "n_visited", "schedulable", "nbt"])
df_piv.columns =list(map(lambda x: "_".join(x), df_piv.columns))
df_piv["n_visted_ratio"] = df_piv["n_visited_ACBFS"]/df_piv["n_visited_BFS"]

sns.displot(data=df_piv, x="n_visted_ratio", kde=True)

In [ ]:
s = df.shape[0] * [True]
df_plot = df.loc[s]
df_piv = df_plot.copy().pivot(index="tid", columns="search_type", values=["duration_s", "n_visited", "schedulable", "nbt"])
df_piv.columns =list(map(lambda x: "_".join(x), df_piv.columns))
df_piv

f, ax = plt.subplots(figsize=(12,12))

sns.scatterplot(data=df_piv, x="duration_s_BFS", y="duration_s_ACBFS", style="schedulable_ACBFS", hue="nbt_BFS", ax=ax, palette=task_palette)
ax.set(xscale="log", yscale="log")
ax.axis('equal')

lims = [
    10**-3.1,
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
]

# now plot both limits against eachother
ax.plot(lims, lims, 'k-', alpha=0.75, zorder=1)

ax.set(xlim=lims, ylim=lims)

In [ ]:
s = df.shape[0] * [True]
df_plot = df.loc[s]
df_piv = df_plot.copy().pivot(index="tid", columns="search_type", values=["duration_s", "n_visited", "schedulable", "nbt"])
df_piv.columns =list(map(lambda x: "_".join(x), df_piv.columns))
df_piv = df_piv.loc[(df_piv["duration_s_ACBFS"]>0) & (df_piv["duration_s_BFS"]>0)]
df_piv["duration_s_ratio"] = df_piv["duration_s_ACBFS"]/df_piv["duration_s_BFS"]

sns.displot(data=df_piv, x="duration_s_ratio", kde=True)

In [ ]:
s = df.shape[0] * [True]

df_plot = df.loc[s]

g=sns.boxplot(data=df_plot, y="n_visited", x="n_task", hue="search_type")
g.set_yscale("log")
plt.show()